In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import numpy as np
import torch.nn.functional as F


train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

In [2]:
# img = torch.squeeze(train_dataset[0][0])
# fig, ax = plt.subplots(nrows=1, ncols=1)
# ax.imshow(img, cmap=plt.cm.gray)
%config Completer.use_jedi = False


In [43]:
#Dataloader 
batch_size = 1
train_loader = DataLoader(
                dataset= train_dataset,
                batch_size=batch_size,
                shuffle=False) 
test_loader = DataLoader(
                dataset = test_dataset,
                batch_size=batch_size,
                shuffle=True)

In [52]:
#setting dtype of the tensors
class NeuralNet():
    def __init__(self, LR, batch_size):
        self.layer1 = np.random.uniform(-0.5, 0.5, (784, 64))
        self.layer2 = np.random.uniform(-0.5, 0.5, (64, 10))
        self.LR = LR
        self.batch_size = batch_size
        
    def softmax(self, x):
        denominator = torch.sum(np.exp(x), 1)
        for i in range(len(x)):
            x[i] = np.divide(x[i], denominator[i])
        return x
    
    def non_linearity(self, x):
        return (1 / (1 - np.exp(-1 * x)))
        
    def forward(self, x):
        x = torch.squeeze(x).view(-1, 784)
        self.input = x.clone()

        x = x @ self.layer1
        x = self.non_linearity(x)
        self.z1 = x.clone()
        x = x @ self.layer2
        x = self.non_linearity(x)
        x = self.softmax(x)
        self.softmax_result = x.clone().float().T
        return x
    
    def backpass(self, loss, outputs, gt):
        d_loss_wrt_preds = (-2/torch.numel(outputs)) * (gt - outputs).float() 
        
        delta_w1 = self.input.T @ d_loss_wrt_preds @ self.der_softmax() @ self.layer2.T

        delta_w2 = self.z1.T.float() @ d_loss_wrt_preds.float() @ self.der_softmax().float() 
        
        self.layer1 = torch.Tensor(self.layer1).float() - self.LR * delta_w1.float()
        self.layer2 = torch.Tensor(self.layer2) - self.LR * delta_w2
    
    def der_softmax(self):
        softmax_d = torch.empty(self.batch_size, len(self.softmax_result), len(self.softmax_result))
        for elem in range()    
            for i in range(softmax_d.shape[1]):
                for j in range(softmax_d.shape[2]):
                    if i == j:
                        softmax_d[i, j] = self.softmax_result[i] * (1 - self.softmax_result[j])
                    else:
                        softmax_d[i, j] = -1 * self.softmax_result[i] * self.softmax_result[j]
        return softmax_d
    
    def one_hot_encoding(self, y):
        return F.one_hot(y, num_classes=10)
    
    def mean_sq_error(self, x, y):
        error = torch.sum((x - y) **2) / torch.numel(x)
        return error 

In [53]:
LR = 0.01
net = NeuralNet(LR, batch_size)
def training(EPOCHS):
    for epoch in range(EPOCHS):
        for x,y in train_loader:
            outputs = net.forward(x)

            gt = net.one_hot_encoding(y)
            batch_loss = net.mean_sq_error(outputs, gt)
            
            net.backpass(loss, outputs, gt)
        total_loss = total_loss + batch_loss 
        print(f"Loss for EPOCH={epoch} is {total_loss}")

training(1)

In [54]:
# Training 

# Testing
def testing(test_loader, net):
    for x, y in test_loader:
        output = net.forward(x)
        prediction = torch.argmax(output)
        print(y, prediction)
        break

torch.Size([1, 10, 10])
torch.Size([1, 10, 10])
torch.Size([1, 10, 10])


RuntimeError: Expected batch2_sizes[0] == bs && batch2_sizes[1] == contraction_size to be true, but got false.  (Could this error message be improved?  If so, please report an enhancement request to PyTorch.)

In [ ]:
testing(test_loader, net)